# AI FAQ 챗봇 만들기 with RAG

In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import json


In [11]:
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

with open('./data/faq_chatbot_data.json', 'r', encoding='utf-8') as f:
    faq_data = json.load(f)

documents = []

for item in faq_data:
    # 각 FAQ 항목을 Document 객체로 변환
    doc = Document(
        page_content=f"질문: {item['question']}\n답변: {item['answer']}",
        metadata={"source": "FAQ"}
    )
    documents.append(doc)

chroma_vector_store = Chroma.from_documents(documents, embedding_model)

retriever = chroma_vector_store.as_retriever()


prompt_template = """
당신은 친절한 FAQ 챗봇입니다. 사용자의 질문에 정확하고 간결하게 답변해 주세요.

FAQ 정보:
{context}

질문: {question}
답변:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# RAG 체인 생성
retrieval_qa = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type='stuff',
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# 대화 시작
print("FAQ 챗봇이 시작되었습니다. 종료하려면 'q' 또는 '종료'를 입력하세요.")

while True:
    # 사용자 입력 받기
    user_input = input("\n질문을 입력하세요: ")
    
    # 종료 조건 확인
    if user_input.lower() in ['q', '종료', 'quit', 'exit']:
        print("챗봇을 종료합니다. 감사합니다!")
        break
    
    # 질문 처리
    try:
        result = retrieval_qa({"query": user_input})
        print("\n답변:", result["result"])
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
        print("다시 시도해 주세요.")

FAQ 챗봇이 시작되었습니다. 종료하려면 'q' 또는 '종료'를 입력하세요.


/var/folders/kp/5_0zhw410tzf9tkb2lt76sw40000gn/T/ipykernel_21216/3295668952.py:64: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = retrieval_qa({"query": user_input})



답변: 답변: 저희 메뉴에는 다양한 맛있는 음식이 있습니다. 추천 메뉴는 고객님의 취향에 따라 다를 수 있으니, 인기 있는 메뉴를 확인해 보시거나 직원에게 문의해 주세요!

답변: 죄송하지만, "Escape"에 대한 정보는 제공할 수 없습니다. 다른 질문이 있으시면 도와드리겠습니다!
챗봇을 종료합니다. 감사합니다!
